# Imports

In [1]:
!/opt/venv/bin/python -m pip install --upgrade pip

In [2]:
!pip install nltk
!pip install spacy-pattern-builder

In [3]:
!pip install spacy
!pip install wikipedia
!python3 -m spacy download en_core_web_lg

     |███████████████████▎            | 498.8 MB 88.4 MB/s eta 0:00:04IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

     |██████████████████████████████▍ | 786.3 MB 2.5 MB/s eta 0:00:17IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [4]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    2.2.4                         
Location         /opt/venv/lib/python3.7/site-packages/spacy
Platform         Linux-4.14.198-152.320.amzn2.x86_64-x86_64-with-debian-10.6
Python version   3.7.9                         
Models                                         



In [5]:
!python3 -m spacy download fr_core_news_sm

     |████████████████████████████████| 14.7 MB 35.7 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


# Look for IS_A pattern


In [6]:
import wikipedia
import spacy
import en_core_web_lg
import fr_core_news_sm
nlp = spacy.load('en_core_web_lg')
nlp_french = fr_core_news_sm.load()

import re 
import string 
import nltk 
import spacy 
import pandas as pd 
import numpy as np 
import math 
from tqdm import tqdm 

from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy 
import random

In [9]:
doc = nlp_french(u"Jarvis Cocker est un chanteur")

for tok in doc: 
    print(tok.text, "-->",tok.dep_, "-->",tok.pos_)

Jarvis --> nsubj --> PROPN
Cocker --> flat:name --> PROPN
est --> cop --> AUX
un --> det --> DET
chanteur --> ROOT --> NOUN


In [10]:
doc = nlp(u"Jarvis Cocker is a singer")

for tok in doc: 
    print(tok.text, "-->",tok.dep_, "-->",tok.pos_)

Jarvis --> compound --> PROPN
Cocker --> nsubj --> PROPN
is --> ROOT --> AUX
a --> det --> DET
singer --> attr --> NOUN


# Partie 2

In [236]:
def get_the_extract(first_sentence):
    new_sentence = re.sub("[\(\[].*?[\)\]]", "", first_sentence)
    new_sentence = new_sentence.replace('  ', ' ')

    doc = nlp(new_sentence)

    word = " " 
    subject = ""
    got_to_root = False

    for i,tok in enumerate(doc): 
        if(tok.dep_ == "nsubj" and not got_to_root):
            # here we go get all the compound possible
            for j in range(0,i):
                if(doc[j].dep_ == "compound"):
                    subject += " " + doc[j].text

            subject += " " + doc[i].text

        if (tok.dep_ == "ROOT"):
            got_to_root = True

        if (tok.dep_ == "attr"):
            word = tok.text

    print(subject, "IS_A", word)

In [12]:
for i in range(1,100):
    
    ran = wikipedia.random()

    try:
        try:
            first_sentence = wikipedia.summary(ran, sentences=1)
        except wikipedia.PageError as e:
            continue
    except wikipedia.DisambiguationError as e:
        continue

    # remove parenthese and it's content
    new_sentence = re.sub("[\(\[].*?[\)\]]", "", first_sentence)
    new_sentence = new_sentence.replace('  ', ' ')

    doc = nlp(new_sentence)
    print("Article: ", i + 1)
    print("Title: ", ran)
    print("ORIGINAL: ", new_sentence)

    word = " " 
    subject = ""
    got_to_root = False

    for i,tok in enumerate(doc): 
        if(tok.dep_ == "nsubj" and not got_to_root):
            # here we go get all the compound possible
            for j in range(0,i):
                if(doc[j].dep_ == "compound"):
                    subject += " " + doc[j].text

            subject += " " + doc[i].text

        if (tok.dep_ == "ROOT"):
            got_to_root = True

        if (tok.dep_ == "attr"):
            word = tok.text

    print(subject, "IS_A", word)
    print("\n")

Article:  2
Title:  List of municipalities of the Province of Syracuse
ORIGINAL:  The following is a list of the 21 municipalities of the Province of Syracuse, Sicily, Italy.
 following IS_A list


Article:  3
Title:  Juglone 3-monooxygenase
ORIGINAL:  In enzymology, juglone 3-monooxygenase is an enzyme that catalyzes the chemical reaction

5-hydroxy-1,4-naphthoquinone + AH2 + O2 
 
  
   
    ⇌
   
  
  {\displaystyle \rightleftharpoons }
  3,5-dihydroxy-1,4-naphthoquinone + A + H2OThe 3 substrates of this enzyme are 5-hydroxy-1,4-naphthoquinone, AH2, and O2, whereas its 3 products are 3,5-dihydroxy-1,4-naphthoquinone, A, and H2O.
This enzyme belongs to the family of oxidoreductases, specifically those acting on paired donors, with O2 as oxidant and incorporation or reduction of oxygen.
 juglone 3-monooxygenase IS_A 3,5-dihydroxy-1,4-naphthoquinone


Article:  4
Title:  Bernd Scholz
ORIGINAL:  Bernd Scholz was a German composer.
 Bernd Scholz IS_A composer


Article:  5
Title:  Latvia

# Partie 3

In [273]:
#define the pattern 

# The NOUN OP + doesn't show more than 1 words at the end, why?


pattern_1 = [{'DEP':'compound', 'OP':"*"},
            {'DEP':'nsubj', 'OP':"+"},
            {'DEP':'prep', 'OP':"?"},
            {'DEP':'pobj', 'OP':"?"},
            {'LEMMA':'be'}, 
            {"TEXT": {"REGEX": "\\ban?\\b"}},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP': 'compound', 'OP':"*"},
            {'DEP': 'attr', 'OP':"+"},
            {'LOWER':'of', 'OP':"?"},
            {'DEP':'det', 'OP':"?"},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP': 'compound', 'OP':"*"},
            {'DEP':'pobj', 'OP':"?"}]

pattern_2 = [{'DEP':'det', 'OP':"*"},
            {'DEP':'compound', 'OP':"*"},
            {'DEP':'nsubj', 'OP':"+"},
            {'DEP':'prep', 'OP':"?"},
            {'DEP':'pobj', 'OP':"?"},
            {'LEMMA':'be'}, 
            {"TEXT": {"REGEX": "\\ban?\\b"}},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP': 'compound', 'OP':"*"},
            {'DEP': 'attr', 'OP':"+"},
            {'LOWER':'of', 'OP':"?"},
            {'DEP':'det', 'OP':"?"},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP': 'compound', 'OP':"*"},
            {'DEP':'pobj', 'OP':"?"}]

pattern_3 = [{'DEP':'det', 'OP':"*"},
            {'DEP':'compound', 'OP':"*"},
            {'DEP':'nsubj', 'OP':"+"},
            {'DEP':'prep', 'OP':"?"},
            {'DEP':'pobj', 'OP':"?"},
            {'LEMMA':'be'}, 
            {"TEXT": {"REGEX": "\\ban?\\b"}},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP': 'compound', 'OP':"+"}]

pattern_4 = [{'POS':'PROPN', 'OP':"*"},
            {'DEP':'compound', 'OP':"*"},
            {'DEP':'nsubj', 'OP':"+"},
            {'DEP':'prep', 'OP':"?"},
            {'DEP':'pobj', 'OP':"?"},
            {'LEMMA':'be'}, 
            {"TEXT": {"REGEX": "\\ban?\\b"}},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP': 'compound', 'OP':"*"},
            {'DEP': 'attr', 'OP':"+"},
            {'LOWER':'of', 'OP':"?"},
            {'DEP':'det', 'OP':"?"},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP': 'compound', 'OP':"*"},
            {'DEP':'pobj', 'OP':"?"}]

pattern_5 = [{'DEP':'det', 'OP':"*"},
            {'DEP':'compound', 'OP':"*"},
            {'DEP':'nsubj', 'OP':"+"},
            {'DEP':'prep', 'OP':"?"},
            {'DEP':'pobj', 'OP':"?"},
            {'LEMMA':'be'}, 
            {"TEXT": {"REGEX": "\\ban?\\b"}},
            {'DEP': 'compound', 'OP':"*"},
            {'DEP': 'attr', 'OP':"+"},
            {'LOWER':'of', 'OP':"?"},
            {'DEP':'det', 'OP':"?"},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP': 'compound', 'OP':"*"},
            {'DEP':'pobj', 'OP':"?"}]




matcher = Matcher(nlp.vocab) 


matcher.add("matching_1", None, pattern_1)
matcher.add("matching_2", None, pattern_2)
matcher.add("matching_3", None, pattern_3)
matcher.add("matching_4", None, pattern_4)
matcher.add("matching_5", None, pattern_5)

In [277]:
#define the pattern 

# The NOUN OP + doesn't show more than 1 words at the end, why?

pattern_1 = [{'DEP':'compound', 'OP':"*"},
            {'DEP':'nsubj', 'OP':"+"},
            {'DEP':'prep', 'OP':"?"},
            {'DEP':'pobj', 'OP':"?"}]

pattern_11 = [{'LEMMA':'be'}, 
            {"TEXT": {"REGEX": "\\ban?\\b"}},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP': 'compound', 'OP':"*"},
            {'DEP': 'attr', 'OP':"+"},
            {'LOWER':'of', 'OP':"?"},
            {'DEP':'det', 'OP':"?"},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP': 'compound', 'OP':"*"},
            {'DEP':'pobj', 'OP':"?"}]

pattern_2 = [{'DEP':'det', 'OP':"*"},
            {'DEP':'compound', 'OP':"*"},
            {'DEP':'nsubj', 'OP':"+"},
            {'DEP':'prep', 'OP':"?"},
            {'DEP':'pobj', 'OP':"?"}]

pattern_22 = [{'LEMMA':'be'}, 
            {"TEXT": {"REGEX": "\\ban?\\b"}},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP': 'compound', 'OP':"*"},
            {'DEP': 'attr', 'OP':"+"},
            {'LOWER':'of', 'OP':"?"},
            {'DEP':'det', 'OP':"?"},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP': 'compound', 'OP':"*"},
            {'DEP':'pobj', 'OP':"?"}]

pattern_3 = [{'DEP':'det', 'OP':"*"},
            {'DEP':'compound', 'OP':"*"},
            {'DEP':'nsubj', 'OP':"+"},
            {'DEP':'prep', 'OP':"?"},
            {'DEP':'pobj', 'OP':"?"}]

pattern_33 = [{'LEMMA':'be'}, 
            {"TEXT": {"REGEX": "\\ban?\\b"}},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP': 'compound', 'OP':"+"}]

pattern_4 = [{'POS':'PROPN', 'OP':"*"},
            {'DEP':'compound', 'OP':"*"},
            {'DEP':'nsubj', 'OP':"+"},
            {'DEP':'prep', 'OP':"?"},
            {'DEP':'pobj', 'OP':"?"}]

pattern_44 = [{'LEMMA':'be'}, 
            {"TEXT": {"REGEX": "\\ban?\\b"}},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP': 'compound', 'OP':"*"},
            {'DEP': 'attr', 'OP':"+"},
            {'LOWER':'of', 'OP':"?"},
            {'DEP':'det', 'OP':"?"},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP': 'compound', 'OP':"*"},
            {'DEP':'pobj', 'OP':"?"}]

pattern_5 = [{'DEP':'det', 'OP':"*"},
            {'DEP':'compound', 'OP':"*"},
            {'DEP':'nsubj', 'OP':"+"},
            {'DEP':'prep', 'OP':"?"},
            {'DEP':'pobj', 'OP':"?"}]

pattern_55 = [{'LEMMA':'be'}, 
            {"TEXT": {"REGEX": "\\ban?\\b"}},
            {'DEP': 'compound', 'OP':"*"},
            {'DEP': 'attr', 'OP':"+"},
            {'LOWER':'of', 'OP':"?"},
            {'DEP':'det', 'OP':"?"},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP': 'compound', 'OP':"*"},
            {'DEP':'pobj', 'OP':"?"}]


matcher_nsubj = Matcher(nlp.vocab) 
matcher_nsubj.add("matching_1", None, pattern_1)
matcher_nsubj.add("matching_2", None, pattern_2)
matcher_nsubj.add("matching_3", None, pattern_3)
matcher_nsubj.add("matching_4", None, pattern_4)
matcher_nsubj.add("matching_5", None, pattern_5)


matcher_root = Matcher(nlp.vocab) 
matcher_root.add("matching_11", None, pattern_11)
matcher_root.add("matching_22", None, pattern_22)
matcher_root.add("matching_33", None, pattern_33)
matcher_root.add("matching_44", None, pattern_44)
matcher_root.add("matching_55", None, pattern_55)

In [262]:
def get_longest_match(matches):
    max_length = 0
    max_id = 0
    
    for i,tup in enumerate(matches):
        if(max_length < (tup[2] - tup[1])):
            max_length = tup[2] - tup[1]
            max_id = i
        # print(doc[matches[i][1]:matches[i][2]])

    span = doc[matches[max_id][1]:matches[max_id][2]]

    return span

In [221]:
def write_in_file(file_name, title, orginal, span):
    string_file = "title : " + title + "\n" + "summary : " + orginal + "\n" + "=> " + str(span) + "\n--\n"
    file_name.write("%s" % string_file)

In [280]:
# matcher = Matcher(nlp.vocab) 
# matcher.add("matching_7", None, pattern_7) 
extract_file = open("extractions3.txt","w+")
# for i in range(1,100):
i = 0
while  i < 100:
    ran = wikipedia.random()
    try:
        try:
            first_sentence = wikipedia.summary(ran, sentences=1)
        except wikipedia.PageError as e:
            continue
    except wikipedia.DisambiguationError as e:
        # still get the error with this ... 
        #s = random.choice(e.options)
        #first_sentence = wikipedia.summary(s, sentences=1)
        continue

    

    # remove parenthese and it's content
    new_sentence = re.sub("[\(\[].*?[\)\]]", "", first_sentence)
    new_sentence = new_sentence.replace('  ', ' ')
    new_sentence = new_sentence.replace('   ', ' ')
    new_sentence = new_sentence.replace('    ', ' ')
    doc = nlp(new_sentence)
    
    print("Article: ", i)
    print("Title: ", ran)
    print("ORIGINAL: ", new_sentence)


    # matches = matcher(doc) 
    matches_nsubj = matcher_nsubj(doc) 
    matches_root = matcher_root(doc) 

    # if len(matches)>0:
    #     i += 1
    #     span = get_longest_match(matches)
    #     print(span)
    #     write_in_file(extract_file, ran, first_sentence, span)

    if len(matches_nsubj)>0 and  len(matches_root)>0:
        i += 1
        span = str(get_longest_match(matches_nsubj)) + ' ' + str(get_longest_match(matches_root))
        print(span)
        write_in_file(extract_file, ran, first_sentence, span)


    print("\n")
    #print(first_sentence)

extract_file.close()

Article:  0
Title:  Barry County Courthouse
ORIGINAL:  The Barry County Courthouse is a government building located at 220 West State Street in Hastings, Michigan.
The Barry County Courthouse is a government building


Article:  1
Title:  Hemberg
ORIGINAL:  Hemberg is a hill ridge in Hesse, Germany.
Hemberg is a hill ridge


Article:  2
Title:  Nerves Junior
ORIGINAL:  Nerves Junior were an American indie rock band based in Louisville, Kentucky.
Nerves Junior were an American indie rock band


Article:  3
Title:  Colegio Hispano Americano de Gunma
ORIGINAL:  Colegio Hispano Americano de Gunma is a Peruvian international school in Isesaki, Gunma, Japan.
Gunma is a Peruvian international school


Article:  4
Title:  Versus (band)
ORIGINAL:  Versus is an American indie rock band formed in 1990 by Richard Baluyut, Fontaine Toups, and Edward Baluyut in New York City, United States.
Versus is an American indie rock band


Article:  5
Title:  Keith Walker (basketball)
ORIGINAL:  Keith B. Walk